# T2.3 - Métodos de clustering espectral


En esta práctica vamos a ver cómo funciona el clustering espectral.

Para ello, como siempre, lo primero es cargar las librerías necesarias:

In [ ]:
import numpy as np
float_formatter = lambda x: "%.3f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
from sklearn.datasets import make_circles
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.metrics import pairwise_distances
from matplotlib import pyplot as plt
import networkx as nx
import seaborn as sns
sns.set()

En nuestro primer ejemplo vamos a usar un dataset muy sencillo que nos permita entender lo que hacemos:

In [ ]:
X = np.array([
    [1, 3], [2, 1], [1, 1],
    [3, 2], [7, 8], [9, 8],
    [9, 9], [8, 7], [13, 14],
    [14, 14], [15, 16], [14, 15]
])
fig, ax = plt.subplots()
ax.scatter(X[:,0], X[:,1])
for i, txt in enumerate(range(X.shape[0])):
    ax.annotate(txt, X[i])
plt.xlabel('Ancho')
plt.ylabel('Largo')

Tal y como acabamos de ver, el agrupamiento espectral consiste en 4 pasos básicos:

-   la obtención de la matriz de adyacencias o afinidad,
-   la obtención de la matriz Laplaciana,
-   el cálculo de los vectores y valores propios de esta última,
-   y el clustering mediante K-means (u otra técnica tradicional).

Vamos a por la primera, la creación de la matriz de adyacencias o afinidad:

In [ ]:
W_dist =## Aquí tu código ##
print(W_dist)

In [ ]:
X.shape

In [ ]:
W_dist.shape

In [ ]:
plt.imshow(W_dist)

In [ ]:
# construímos la matriz de adyacencias o afinidad
W_ad = np.zeros_like(W_dist, np.uint8)
W_ad[W_dist < 5 ] = 1
print(W_ad)

Vamos a utilizar el paquete networkx para visualizar el grafo

In [ ]:
def draw_graph(G):
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos)
    nx.draw_networkx_labels(G, pos)
    nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)

In [ ]:
G = nx.DiGraph(np.array(W_ad))
draw_graph(G)

`networkx` también nos permite calcular la matriz de afinidad a partir del grafo. Vamos a ver como:

In [ ]:
W_ad = nx.adjacency_matrix(G)
print(W_ad)

Fijaos que esto que nos devuelve no parece una matriz, verdad? Habéis oído hablar del término *sparse*? Pues esto que acabamos de ver es una matriz *sparse*, que simplemente es una matriz con muy pocos elementos diferentes de 0.

Para verla en formato matriz necesitamos hacer uso del método `todense()`

In [ ]:
print(W_ad.todense())

Ahora tenemos que obtener la matriz Laplaciana, que recordad que se obtenía como la resta entre la matriz de adyacencia y la matriz de grado.

In [ ]:
# matriz de grado
D = ## Aquí tu código ##
print('Matriz de grado:\n', D)

In [ ]:
# matriz laplaciana
L = ## Aquí tu código ##
print('Matriz Laplaciana:\n', L)

La matriz Laplaciana es una matriz especial con ciertas propiedades que nos facilitan la vida. Una de ellas es que:

-   **si el grafo `W` tiene `K` componentes conexas, entonces `L` tiene `K` vectores propios (*eigenvectors*) con valor propio (*eigenvalue*) igual a 0.**

En este caso, cuantos eigenvectors vamos a tener iguales a 0?

In [ ]:
e, v = ## Aquí tu código ##
# eigenvalues
print('eigenvalues:')
print(e)
# eigenvectors
print('eigenvectors:')
print(v)

En efecto, 3! No nos debería sorprender, ya que nuestro conjunto de datos tiene 3 clusters conexos.

In [ ]:
plt.figure()
plt.plot(e)
plt.title('eigenvalues')

eigen_0 = np.where(e < 10e-6)[0]
print(f'Tenemos {len(eigen_0)} componentes conexos en nuestro dataset.')
for n, i in enumerate(eigen_0):
    plt.figure()
    plt.plot(v[:, i])
    plt.title(f'{n} eigenvector con eigenvalue=0')

In [ ]:
# veamos nuestro dataset de nuevo
fig, ax = plt.subplots()
ax.scatter(X[:,0], X[:,1], alpha=0.7, edgecolors='b')
for i, txt in enumerate(range(X.shape[0])):
    ax.annotate(txt, X[i])
plt.xlabel('Ancho')
plt.ylabel('Largo')

Si nos fijamos en las gráficas de los 3 primeros vectores propios de la matriz L (cuyos valores propios son 0), podemos ver cómo nos permiten separar correctamente los 3 clusters de nuestro dataset trazando una línea horizontal.

Recordad que esto es buena señal, nos indica que con esta nueva representación de los datos, un algoritmo de clustering podrá separarlos correctamente.

Vamos a comprobar si es cierto:

In [ ]:
eigen_0

In [ ]:
vectores_propios_a_utilizar = eigen_0 # en este caso queremos usar los 3, podría ser diferente

Xnew = np.array(v[:, vectores_propios_a_utilizar])
print(Xnew.shape)
km = KMeans(init='k-means++', n_clusters=len(eigen_0))
km.fit(Xnew)
km.labels_

In [ ]:
# veamos el resultado del clustering
fig, ax = plt.subplots()
ax.scatter(X[:,0], X[:,1], c=km.labels_)
for i, txt in enumerate(range(X.shape[0])):
    ax.annotate(txt, X[i])
plt.xlabel('Ancho')
plt.ylabel('Largo')

¿Y si visualizamos los datos proyectados usando los 3 vectores propios?

In [ ]:
# veamos el dataset proyectado usando los 3 vectores propios escogidos
fig, ax = plt.subplots()
ax.scatter(Xnew[:, 0], Xnew[:, 1], c=km.labels_)
plt.xlabel('Ancho')
plt.ylabel('Largo')

Fijaos lo fácil que debe ser ahora para el k-means con esta nueva representación de los datos.

Como habréis podido comprobar, al final, lo único que estamos haciendo es cambiar el espacio de representación de los datos por uno que nos permita agruparlos mejor. ¿Os suena esto de algo?

En efecto, es básicamente el modo de funcionamiento de las redes neuronales profundas, en las que los datos van sufriendo transformaciones en cada capa hasta llegar a una representación que facilita la tarea a desarrollar.

"Pero, este ejemplo es muy sencillo", podéis estar pensando. Y no os falta razón. Vamos a complicarlo un poco:

In [ ]:
X, clusters = make_circles(n_samples=1000, noise=.05, factor=.5, random_state=0)
print(X.shape)
plt.scatter(X[:,0], X[:,1])

Vamos a probar con el k-means, a ver qué tal...

In [ ]:
km = ## Aquí tu código ##
km_clustering = km.fit(X)
plt.scatter(X[:,0], X[:,1], c=km_clustering.labels_)

Parece que no os he engañado, el k-means no es capaz de hacer el agrupamiento de forma correcta.

Vamos a ver si con lo que acabamos de aprender somos capaces. Para ello, recordad que necesitamos:

-   la obtención de la matriz de adyacencias o afinidad,
-   la obtención de la matriz Laplaciana,
-   el cálculo de los vectores y valores propios de esta última,
-   transformación de los datos en el nuevo espacio
-   y el clustering mediante K-means (u otra técnica tradicional).

Manos a la obra:

In [ ]:
W_dist = ## Aquí tu código ##
print('Matriz distancias:\n', W_dist)
plt.imshow(W_dist)
plt.colorbar()
plt.grid(False)

In [ ]:
# Vamos a escoger 0.5 como el umbral
W_ad = np.zeros_like(W_dist, np.uint8)
W_ad[## Aquí tu código ##] = 1
print('Matriz de adyacencia:\n', W_ad)

In [ ]:
# matriz de grado
D = ## Aquí tu código ##
print('Matriz de grado:\n', D)

In [ ]:
# matriz laplaciana
L = ## Aquí tu código ##
print('Matriz Laplaciana:\n', L)

In [ ]:
# calculamos los eigenvectors y eigenvalues
e, v = ## Aquí tu código ##

In [ ]:
plt.figure()
plt.plot(e)
plt.title('eigenvalues')

eigen_0 = np.isclose(e, 0, atol=1e-3)
print(f'Tenemos {sum(eigen_0)} componentes conexos en nuestro dataset.')

Fijaos que mirando los eigenvalues no parece que sea fácil elegir con qué componentes queremos quedarnos para hacer el clustering, ¿verdad?

¿Se os ocurre alguna forma mejor de seguir?

¿Probamos a calcular la matriz laplaciana normalizada simétrica a ver si mejora?

In [ ]:
# calculamos la laplaciana normalizada simétrica
D = np.sum(W_ad, axis=1)
D = D**(-1./2)
I = np.diag(np.ones(D.size))
D = np.diag(D)
L = I-D.dot(W_ad).dot(D)

# calculamos autovectores y autovalores
e, v = ## Aquí tu código ##
e = e.real
v = v.real


plt.figure()
plt.plot(e)
plt.title('eigenvalues')

eigen_0 = np.where(np.isclose(e, 0, atol=1e-1))[0]
print(f'Tenemos {len(eigen_0)} componentes conexos en nuestro dataset.')
for n, i in enumerate(eigen_0):
    plt.figure()
    plt.plot(v[:, i])
    plt.title(f'{n} eigenvector con eigenvalue=0')

En este caso parece que encuentra las 3 componentes conexas! Vamos a ver si funciona el clustering:

In [ ]:
vectores_propios_a_utilizar = eigen_0
print(vectores_propios_a_utilizar)
Xnew = np.array(v[:, vectores_propios_a_utilizar])
km = KMeans(init='k-means++', n_clusters=2)
km.fit(Xnew)

In [ ]:
# veamos el resultado del clustering
fig, ax = plt.subplots()
ax.scatter(X[:,0], X[:,1], c=km.labels_)

Parece que no. Vamos a ver los datos en su nuevo espacio de representación:

In [ ]:
# veamos el dataset proyectado usando los 3 vectores propios escogidos
fig, ax = plt.subplots()
ax.scatter(Xnew[:,0], Xnew[:,1], c=km.labels_, alpha=0.7)

Fijaos que mirando la nueva representación de los datos, no parece que el k-means vaya a poder hacer bien el clustering, ¿verdad?

¿Se os ocurre alguna forma mejor de seguir?

¿Quizá pensando en cómo hemos construído la matriz de adyacencia? ¿Existe alguna otra forma de hacerlo?

Eso es, podemos intentarlo haciendo uso del kNN en vez del umbral ($\epsilon$).

Vamos allá:

In [ ]:
def matriz_afinidad_KNN(mSimilitud, KNN=5):
    auxM = mSimilitud.copy()
    np.fill_diagonal(auxM, 0)

    # Construimos la matriz afinidad de A a B
    mAfinidadA = np.zeros(auxM.shape)
    # utilizamos la matriz similitud para ordenar los vecinos de cada nodo por cercanía
    # `np.argsort` nos devuelve los índices que ordenan el array introducido (de forma ascendente)
    # en este caso, al tener la matriz similitud, tenemos que hacerla negativa, para que nos
    # devuelva primero los más similares (que al ponerle el - a la matriz de similitud, son los
    # valores más pequeños)
    # Flatten simplemente lo utilizamos para convertir la matriz `KNN x n`en un vector de 1x(KNN x n)
    indices_kNN_nodo_fila = np.argsort(-auxM, axis=0)[0:KNN, :].flatten()
    # Nos creamos un array para combinar con el anterior y poder indexar los
    # elementos pertinentes de la matriz de afinidad
    indices_kNN_nodo_col = np.tile(np.arange(auxM.shape[0]), KNN)
    mAfinidadA[indices_kNN_nodo_fila, indices_kNN_nodo_col] = 1
    np.fill_diagonal(mAfinidadA, 1)

    # Y ahora hacemos lo mismo de B a A
    mAfinidadB = np.zeros(auxM.shape)
    # Fijaos que en este caso la matriz va a ser `n x KNN` (hacemos el argsort en la dirección columnas: -->)
    indices_kNN_nodo_fila = np.repeat(np.arange(auxM.shape[0]), KNN)
    indices_kNN_nodo_col = np.argsort(-auxM, axis=1)[:, 0:KNN].flatten()
    mAfinidadB[indices_kNN_nodo_fila, indices_kNN_nodo_col] = 1
    np.fill_diagonal(mAfinidadB, 1)

    return (mAfinidadA + mAfinidadB) / 2

In [ ]:
# calculamos de nuevo la matriz de adyacencia (o afinidad), esta vez siguiendo el enfoque de los kNN
sigma = 0.1
W_sim = np.exp(-np.power(W_dist,2)/(2*sigma**2))
W_ad = matriz_afinidad_KNN(W_sim)
print(W_ad)

In [ ]:
# calculamos la laplaciana normalizada simétrica
D = np.sum(W_ad,axis=1)
D = D**(-1./2)
I = np.diag(np.ones(D.size))
D = np.diag(D)
L = I - D.dot(W_ad).dot(D)

# calculamos autovectores y autovalores
e, v = np.linalg.eig(L)

plt.figure()
plt.plot(e)
plt.title('eigenvalues')

eigen_0 = np.where(np.isclose(e, 0, atol=1e-5))[0]
print(f'Tenemos {len(eigen_0)} componentes conexos en nuestro dataset.')
for n, i in enumerate(eigen_0):
    plt.figure()
    plt.plot(v[:, i])
    plt.title(f'{n} eigenvector con eigenvalue=0')

Parece que ahora encuentra 2 componentes conexos! Vamos a probar a hacer el clustering:

In [ ]:
vectores_propios_a_utilizar = eigen_0
print(vectores_propios_a_utilizar)
Xnew = np.array(v[:, vectores_propios_a_utilizar])
km = ## Aquí tu código ##
km.fit(Xnew)

Veamos el resultado:

In [ ]:
# veamos el resultado del clustering
fig, ax = plt.subplots()
ax.scatter(X[:,0], X[:,1], c=km.labels_, cmap='jet')

Fijaos como es muy importante elegir el tipo de enfoque adecuado dependiendo de los datos. Cuando hemos construido la $W_{ad}$ mediante el método del umbral, nos hemos inventado completamente dicho umbral.

Esto no quiere decir que no exista un umbral para el cual no funcione correctamente el clustering siguiendo ese enfoque, lo que quiere decir es que si no conocemos el umbral, es posiblemente más sencillo seguir el enfoque de los kNN.

Vamos a ver los datos en su nuevo espacio:

In [ ]:
# veamos el dataset proyectado usando los 2 vectores propios escogidos
fig, ax = plt.subplots()
ax.scatter(Xnew[:,0], Xnew[:,1], c=km.labels_, alpha=0.7, cmap='jet')

Maravilloso, ¿no os parece?


Y más maravilloso os va a parecer aún lo siguiente:

In [ ]:
sc = SpectralClustering(n_clusters=2, affinity='nearest_neighbors', random_state=0)
sc_clustering = sc.fit(X)
plt.scatter(X[:,0], X[:,1], c=sc_clustering.labels_, cmap='jet')

En efecto, así de rápido podemos realizar un clustering espectral utilizando las librerías existentes!

Fuente: https://towardsdatascience.com/unsupervised-machine-learning-spectral-clustering-algorithm-implemented-from-scratch-in-python-205c87271045

Ejemplo paso a paso para entender y profundizar:
-  https://towardsdatascience.com/spectral-clustering-aba2640c0d5b

Más ejemplos:

-  https://scikit-learn.org/stable/auto_examples/cluster/plot_segmentation_toy.html
-  https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_segmentation.html

Lecturas discutiendo la similitud de k-means, clustering espectral y PCA, para quien tenga interés:

-  https://stats.stackexchange.com/a/151665
-  https://stats.stackexchange.com/a/189324
-  https://stats.stackexchange.com/a/189324 (muy completa y didáctica)
-  https://qr.ae/TW25h8
